In [2]:
%pip install -r requirements.txt

import pandas as pd
import numpy as np


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: C:\Users\Admin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [3]:
df = pd.read_csv('./data/interest_rate.csv')
df['year'] = pd.DatetimeIndex(df['TIME']).year
df['month'] = pd.DatetimeIndex(df['TIME']).month

df = df.drop('TIME', axis=1)

longTerm = df.query('Subject == "Long-term interest rates, Per cent per annum"')
shortTerm = df.query('Subject == "Short-term interest rates, Per cent per annum"')
immediateTerm = df.query('Subject == "Immediate interest rates, Call Money, Interbank Rate, Per cent per annum"')

longTerm.to_csv('./data/longTerm.csv', index=False)
shortTerm.to_csv('./data/shortTerm.csv', index=False)
immediateTerm.to_csv('./data/immediateTerm.csv', index=False)